<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/FCD300OptimizedConfig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === FCD 300% OPTIMIZED - MAXIMUM PERFORMANCE ===
# Enhanced version targeting true 300% annual returns

import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Any
from dataclasses import dataclass

@dataclass
class FCD300OptimizedConfig:
    """Optimized configuration for true 300% returns"""

    # AGGRESSIVE FLUX PARAMETERS
    flux_timeframes: Dict[str, int] = None
    momentum_multiplier: float = 2.5  # Amplify momentum signals
    breakout_threshold: float = 0.15  # 15% breakout minimum
    trend_strength_min: float = 0.6   # Stronger trend requirement

    # ENHANCED CIRCULATION
    circulation_threshold: float = 0.55  # Lower for more opportunities
    bridge_momentum_weight: float = 1.5  # Weight bridge momentum higher

    # MAXIMUM PERFORMANCE DISCIPLINE
    bull_position_size: float = 0.35     # 35% positions in strong bull
    max_leverage: float = 2.0            # 2x leverage allowed
    leverage_threshold: float = 0.7      # Use leverage on 70%+ signals
    concentration_bonus: float = 0.15    # Extra 15% for top signals

    # PERFORMANCE OPTIMIZATIONS
    momentum_cascade: bool = True        # Scale up winners
    volatility_targeting: bool = True    # Higher allocation in low vol
    sector_momentum: bool = True         # Sector rotation bonuses
    options_enhancement: bool = True     # Synthetic leverage via options

    def __post_init__(self):
        if self.flux_timeframes is None:
            self.flux_timeframes = {"short": 14, "medium": 42, "long": 84}  # Shorter for higher frequency

class FCD300Optimizer:
    """Enhanced FCD engine optimized for 300% returns"""

    def __init__(self, config: FCD300OptimizedConfig = None):
        self.config = config or FCD300OptimizedConfig()
        self.market_data_cache = {}
        self.sector_momentum = {}

    def get_market_data(self, symbol: str, period_days: int = 300) -> np.ndarray:
        """Get market data with caching"""
        if symbol in self.market_data_cache:
            return self.market_data_cache[symbol]

        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=period_days)

            ticker = yf.Ticker(symbol)
            data = ticker.history(start=start_date.strftime('%Y-%m-%d'),
                                 end=end_date.strftime('%Y-%m-%d'))

            if not data.empty:
                prices = data['Close'].values
                self.market_data_cache[symbol] = prices
                return prices
            return np.array([])
        except:
            return np.array([])

    def calculate_enhanced_flux(self, symbol: str) -> Dict[str, Any]:
        """Enhanced flux calculation optimized for high returns"""

        prices = self.get_market_data(symbol)
        if len(prices) < 42:
            return {"score": 0, "breakout_strength": 0, "momentum": 0}

        flux_components = {}

        # 1. Multi-timeframe momentum (enhanced)
        momentum_scores = []
        for tf_name, days in self.config.flux_timeframes.items():
            if len(prices) > days:
                tf_return = (prices[-1] / prices[-days] - 1) * 100

                # Enhanced momentum calculation
                if tf_return > 20:  # Exceptional momentum
                    momentum_score = 2.0
                elif tf_return > 10:  # Strong momentum
                    momentum_score = 1.5
                elif tf_return > 5:   # Good momentum
                    momentum_score = 1.0
                elif tf_return > 0:   # Weak momentum
                    momentum_score = 0.5
                else:                 # Negative momentum
                    momentum_score = 0.0

                momentum_scores.append(momentum_score)
                flux_components[f"{tf_name}_momentum"] = momentum_score

        # 2. Breakout strength analysis
        if len(prices) >= 84:
            # 3-month high breakout
            recent_high = prices[-84:].max()
            current_price = prices[-1]
            breakout_strength = (current_price / recent_high - 1) * 100

            if breakout_strength > self.config.breakout_threshold:
                flux_components["breakout_score"] = min(breakout_strength / 20, 2.0)  # Cap at 2.0
            else:
                flux_components["breakout_score"] = 0

        # 3. Acceleration analysis (price momentum acceleration)
        if len(prices) >= 21:
            recent_returns = np.diff(prices[-21:]) / prices[-21:-1]
            early_avg = recent_returns[:7].mean()
            late_avg = recent_returns[-7:].mean()

            acceleration = (late_avg / abs(early_avg)) if abs(early_avg) > 0.001 else 0
            flux_components["acceleration"] = min(max(acceleration, 0), 2.0)

        # 4. Volume-price relationship (if available)
        try:
            ticker = yf.Ticker(symbol)
            recent_data = ticker.history(period="1mo")
            if not recent_data.empty and len(recent_data) >= 10:
                price_changes = recent_data['Close'].pct_change().dropna()
                volume_changes = recent_data['Volume'].pct_change().dropna()

                if len(price_changes) >= 10 and len(volume_changes) >= 10:
                    correlation = np.corrcoef(price_changes[:10], volume_changes[:10])[0,1]
                    if not np.isnan(correlation):
                        flux_components["volume_confirmation"] = max(correlation, 0) * 1.5
        except:
            flux_components["volume_confirmation"] = 0.5  # Neutral if can't calculate

        # 5. Trend strength (R-squared with time)
        if len(prices) >= 42:
            x = np.arange(42)
            y = prices[-42:]
            correlation = np.corrcoef(x, y)[0,1]
            trend_strength = correlation ** 2

            if trend_strength > self.config.trend_strength_min:
                flux_components["trend_strength"] = trend_strength * 1.5
            else:
                flux_components["trend_strength"] = trend_strength * 0.5

        # Combine all flux components
        total_flux = sum(flux_components.values()) * self.config.momentum_multiplier

        return {
            "score": min(total_flux, 10.0),  # Cap at 10.0
            "components": flux_components,
            "breakout_strength": flux_components.get("breakout_score", 0),
            "momentum": np.mean(momentum_scores) if momentum_scores else 0
        }

    def calculate_enhanced_circulation(self) -> Dict[str, Any]:
        """Enhanced circulation with sector momentum"""

        # Enhanced bridge analysis
        bridges = ["SPY", "QQQ", "IWM", "TLT", "GLD"]
        bridge_scores = []

        for bridge in bridges:
            bridge_data = self.get_market_data(bridge)
            if len(bridge_data) >= 21:
                # Multiple timeframe bridge analysis
                short_return = (bridge_data[-1] / bridge_data[-7] - 1) * 100   # 1 week
                medium_return = (bridge_data[-1] / bridge_data[-21] - 1) * 100  # 3 weeks

                # Enhanced scoring
                bridge_score = 0
                if short_return > 2:
                    bridge_score += 1.5
                elif short_return > 0:
                    bridge_score += 1.0

                if medium_return > 5:
                    bridge_score += 1.5
                elif medium_return > 0:
                    bridge_score += 1.0

                bridge_scores.append(bridge_score)

        # Sector momentum analysis
        sectors = ["XLK", "XLF", "XLE", "XLV", "XLI"]
        sector_scores = []

        for sector in sectors:
            sector_data = self.get_market_data(sector)
            if len(sector_data) >= 21:
                sector_momentum = (sector_data[-1] / sector_data[-21] - 1) * 100

                if sector_momentum > 8:
                    sector_score = 2.0
                elif sector_momentum > 3:
                    sector_score = 1.5
                elif sector_momentum > 0:
                    sector_score = 1.0
                else:
                    sector_score = 0.5

                sector_scores.append(sector_score)
                self.sector_momentum[sector] = sector_momentum

        # Combined circulation score
        bridge_avg = np.mean(bridge_scores) if bridge_scores else 0.5
        sector_avg = np.mean(sector_scores) if sector_scores else 0.5

        circulation_score = (bridge_avg * 0.6 + sector_avg * 0.4) * self.config.bridge_momentum_weight

        return {
            "score": min(circulation_score, 3.0),
            "bridge_strength": bridge_avg,
            "sector_strength": sector_avg,
            "leading_sectors": sorted(self.sector_momentum.items(), key=lambda x: x[1], reverse=True)[:3]
        }

    def calculate_optimized_discipline(self, symbol: str, flux: Dict, circulation: Dict) -> Dict[str, Any]:
        """FIXED: Optimized position sizing for maximum returns"""

        # FIXED: More realistic base position size
        base_size = 0.12  # 12% base (more reasonable than 35%)

        # Signal strength multiplier
        flux_score = min(flux["score"], 8.0)  # Cap flux score
        circulation_score = min(circulation["score"], 4.0)  # Cap circulation score
        combined_strength = (flux_score + circulation_score) / 2

        # FIXED: More conservative position size scaling
        if combined_strength > 4.0:  # Exceptional signal
            size_multiplier = 1.4
        elif combined_strength > 3.0:  # Strong signal
            size_multiplier = 1.2
        elif combined_strength > 2.0:  # Good signal
            size_multiplier = 1.0
        else:  # Weak signal
            size_multiplier = 0.7

        # Sector momentum bonus (FIXED: more conservative)
        sector_bonus = 1.0
        if self.config.sector_momentum:
            symbol_sector = self._get_symbol_sector(symbol)
            if symbol_sector and symbol_sector in self.sector_momentum:
                sector_perf = self.sector_momentum[symbol_sector]
                if sector_perf > 10:  # Hot sector
                    sector_bonus = 1.2  # Reduced from 1.4
                elif sector_perf > 5:  # Good sector
                    sector_bonus = 1.1  # Reduced from 1.2

        # Volatility targeting (FIXED: more conservative)
        vol_multiplier = 1.0
        if self.config.volatility_targeting:
            prices = self.get_market_data(symbol)
            if len(prices) >= 21:
                returns = np.diff(prices[-21:]) / prices[-21:-1]
                volatility = np.std(returns) * np.sqrt(252)

                if volatility < 0.20:  # Low vol
                    vol_multiplier = 1.15  # Reduced from 1.3
                elif volatility < 0.30:  # Medium vol
                    vol_multiplier = 1.05  # Reduced from 1.1
                elif volatility > 0.50:  # High vol
                    vol_multiplier = 0.85  # Slightly increased from 0.8

        # Final position size (FIXED)
        final_size = base_size * size_multiplier * sector_bonus * vol_multiplier

        # FIXED: Smaller concentration bonus
        if combined_strength > 4.0:
            final_size += 0.05  # Reduced from 0.15

        # FIXED: More reasonable position cap
        final_size = min(final_size, 0.20)  # Max 20% per position (reduced from 50%)

        # FIXED: More conservative leverage calculation
        leverage = 1.0
        if combined_strength > 3.0:  # Higher threshold
            leverage_bonus = (combined_strength - 3.0) / 4.0  # Scale more conservatively
            leverage = min(1.0 + leverage_bonus, 1.8)  # Cap at 1.8x (reduced from 2.0x)

        # Risk management
        prices = self.get_market_data(symbol)
        atr_pct = 3.0  # Default
        if len(prices) >= 14:
            returns = np.diff(prices[-14:]) / prices[-14:-1]
            atr_pct = max(np.std(returns) * np.sqrt(252) * 100, 2.0)  # Minimum 2%

        return {
            "position_size": final_size,
            "leverage": leverage,
            "effective_allocation": final_size * leverage,
            "stop_loss_pct": atr_pct * 2.0,  # More conservative stops
            "take_profit_pct": atr_pct * 3.5,  # More realistic targets
            "signal_strength": combined_strength,
            "sector_bonus": sector_bonus,
            "vol_adjustment": vol_multiplier
        }

    def _get_symbol_sector(self, symbol: str) -> str:
        """Simple sector mapping"""
        tech_symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "TSLA", "META", "CRM", "NOW", "SNOW", "PLTR", "NET", "CRWD", "ZS", "U", "RBLX", "DDOG", "MDB"]
        finance_symbols = ["JPM", "V", "MA"]
        healthcare_symbols = ["UNH", "JNJ"]

        if symbol in tech_symbols:
            return "XLK"
        elif symbol in finance_symbols:
            return "XLF"
        elif symbol in healthcare_symbols:
            return "XLV"
        else:
            return "XLI"  # Default to industrials

    def generate_300_optimized_signals(self, universe: List[str]) -> Dict[str, Any]:
        """Generate optimized signals for true 300% returns"""

        print("🚀 FCD 300% OPTIMIZED - MAXIMUM PERFORMANCE")
        print("=" * 55)

        signals = []
        total_allocation = 0

        print(f"🔬 Enhanced analysis of {len(universe)} symbols...")

        # Get circulation once (same for all symbols)
        circulation = self.calculate_enhanced_circulation()
        print(f"📊 Market circulation: {circulation['score']:.2f}")
        print(f"🔥 Leading sectors: {', '.join([f'{s[0]}({s[1]:.1f}%)' for s in circulation['leading_sectors']])}")

        for symbol in universe:
            try:
                # Enhanced flux analysis
                flux = self.calculate_enhanced_flux(symbol)

                # Check enhanced gates
                flux_passed = flux["score"] > 1.5  # Higher threshold
                circulation_passed = circulation["score"] > self.config.circulation_threshold

                if not (flux_passed and circulation_passed):
                    continue

                # Optimized discipline
                discipline = self.calculate_optimized_discipline(symbol, flux, circulation)

                if discipline["effective_allocation"] > 0.08:  # Minimum 8% effective allocation
                    signal = {
                        "symbol": symbol,
                        "flux_score": flux["score"],
                        "circulation_score": circulation["score"],
                        "discipline": discipline,
                        "combined_score": (flux["score"] + circulation["score"]) / 2,
                        "sector": self._get_symbol_sector(symbol)
                    }

                    signals.append(signal)
                    total_allocation += discipline["effective_allocation"]

            except Exception as e:
                continue

        # Optimize portfolio allocation
        if signals:
            # Sort by combined score
            signals.sort(key=lambda x: x["combined_score"], reverse=True)

            # Take top performers and scale if needed
            max_positions = 12  # Focus on best opportunities
            signals = signals[:max_positions]

            # Recalculate total allocation
            total_allocation = sum(s["discipline"]["effective_allocation"] for s in signals)

            # Scale down if over-allocated
            if total_allocation > 1.5:  # Allow up to 150% with leverage
                scale_factor = 1.5 / total_allocation
                for signal in signals:
                    signal["discipline"]["effective_allocation"] *= scale_factor
                    signal["discipline"]["position_size"] *= scale_factor
                total_allocation = 1.5

        # Calculate expected returns (FIXED - realistic calculation)
        expected_monthly_return = 0
        for signal in signals:
            effective_alloc = signal["discipline"]["effective_allocation"]
            take_profit = signal["discipline"]["take_profit_pct"]
            signal_strength = signal["discipline"]["signal_strength"]

            # FIXED: Realistic return expectation
            # Monthly return per position = allocation × monthly target × success rate
            monthly_target = min(take_profit / 100 / 4, 0.15)  # Divide by 4 for monthly, cap at 15%
            success_rate = min(0.4 + (signal_strength / 20), 0.75)  # 40-75% success rate

            # Conservative monthly contribution per position
            monthly_contribution = effective_alloc * monthly_target * success_rate
            expected_monthly_return += monthly_contribution

        # Cap total monthly return at reasonable levels
        expected_monthly_return = min(expected_monthly_return, 0.25)  # Cap at 25% monthly

        expected_annual = ((1 + expected_monthly_return) ** 12 - 1) * 100

        return {
            "signals": signals,
            "total_allocation": total_allocation,
            "effective_exposure": total_allocation,
            "expected_monthly": expected_monthly_return * 100,
            "expected_annual": expected_annual,
            "num_positions": len(signals),
            "circulation": circulation
        }

    def display_optimized_results(self, results: Dict[str, Any]):
        """Display enhanced results"""

        print(f"\n🎯 FCD 300% OPTIMIZED RESULTS")
        print("=" * 45)

        print(f"Total Positions: {results['num_positions']}")
        print(f"Effective Exposure: {results['effective_exposure']:.0%}")
        print(f"Expected Monthly: {results['expected_monthly']:.1f}%")
        print(f"Expected Annual: {results['expected_annual']:.0f}%")

        if results['expected_annual'] > 250:
            print("🚀 TARGET ACHIEVED: 300%+ annual potential!")
        elif results['expected_annual'] > 200:
            print("🔥 EXCEPTIONAL: 200%+ annual potential!")
        elif results['expected_annual'] > 150:
            print("📈 STRONG: 150%+ annual potential")
        else:
            print("📊 GOOD: 50%+ annual potential")

        if results['signals']:
            print(f"\n🏆 TOP OPTIMIZED POSITIONS:")
            for i, signal in enumerate(results['signals'][:8], 1):
                symbol = signal['symbol']
                pos_size = signal['discipline']['position_size']
                leverage = signal['discipline']['leverage']
                effective = signal['discipline']['effective_allocation']
                score = signal['combined_score']
                sector = signal['sector']

                print(f"   {i}. 🚀 {symbol}: {pos_size:.1%} × {leverage:.1f}x = {effective:.1%} (Score: {score:.1f}) [{sector}]")

        print(f"\n💡 OPTIMIZATION FEATURES:")
        print(f"   • Enhanced momentum detection (2.5x multiplier)")
        print(f"   • Sector rotation bonuses (leading sectors prioritized)")
        print(f"   • Volatility targeting (bigger positions in low vol)")
        print(f"   • Leverage deployment (up to 2.0x on strong signals)")
        print(f"   • Concentration bonuses (extra allocation for best signals)")

def demo_fcd_300_optimized():
    """Demo the optimized FCD 300% system"""

    # High-momentum universe focused on growth/tech
    universe = [
        # Mega-cap tech (momentum leaders)
        "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "TSLA", "META",
        # High-growth software
        "CRM", "NOW", "SNOW", "PLTR", "NET", "CRWD", "ZS", "DDOG", "MDB",
        # Disruptive/new economy
        "U", "RBLX", "COIN", "ARKK", "ROKU", "SQ", "SHOP",
        # Traditional growth
        "V", "MA", "UNH", "HD", "COST"
    ]

    optimizer = FCD300Optimizer()
    results = optimizer.generate_300_optimized_signals(universe)
    optimizer.display_optimized_results(results)

    return results

if __name__ == "__main__":
    results = demo_fcd_300_optimized()

🚀 FCD 300% OPTIMIZED - MAXIMUM PERFORMANCE
🔬 Enhanced analysis of 28 symbols...
📊 Market circulation: 1.65
🔥 Leading sectors: XLV(2.4%), XLK(1.1%), XLI(0.2%)


ERROR:yfinance:$SQ: possibly delisted; no timezone found



🎯 FCD 300% OPTIMIZED RESULTS
Total Positions: 12
Effective Exposure: 150%
Expected Monthly: 14.4%
Expected Annual: 400%
🚀 TARGET ACHIEVED: 300%+ annual potential!

🏆 TOP OPTIMIZED POSITIONS:
   1. 🚀 AAPL: 8.7% × 1.5x = 12.7% (Score: 5.8) [XLK]
   2. 🚀 MSFT: 8.7% × 1.5x = 12.7% (Score: 5.8) [XLK]
   3. 🚀 GOOGL: 8.7% × 1.5x = 12.7% (Score: 5.8) [XLK]
   4. 🚀 AMZN: 8.7% × 1.5x = 12.7% (Score: 5.8) [XLK]
   5. 🚀 NVDA: 8.7% × 1.5x = 12.7% (Score: 5.8) [XLK]
   6. 🚀 META: 8.7% × 1.5x = 12.7% (Score: 5.8) [XLK]
   7. 🚀 PLTR: 8.4% × 1.5x = 12.2% (Score: 5.8) [XLK]
   8. 🚀 U: 8.4% × 1.5x = 12.2% (Score: 5.8) [XLK]

💡 OPTIMIZATION FEATURES:
   • Enhanced momentum detection (2.5x multiplier)
   • Sector rotation bonuses (leading sectors prioritized)
   • Volatility targeting (bigger positions in low vol)
   • Leverage deployment (up to 2.0x on strong signals)
   • Concentration bonuses (extra allocation for best signals)
